In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

spark = SparkSession.builder.appName('lrex').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/03 19:07:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
actualData = spark.read.csv(r'../../Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv', header=True, inferSchema=True)

In [8]:
actualData.show(5)
actualData.printSchema()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [9]:
actualData.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [11]:
# Making the DataFrame for ML and MLlib 
# We will try to grab few columns as feature and one a label that we are trying to predict

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'], outputCol='features')
outputData = assembler.transform(actualData)

In [14]:
outputData.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [17]:
finalOpData = outputData.select(['features', 'Yearly Amount Spent'])
finalOpData.show(5)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
+--------------------+-------------------+
only showing top 5 rows



In [18]:
# Making the Training and the Test Dataset
trainingData, testData = finalOpData.randomSplit([0.7, 0.3])

In [20]:
trainingData.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[29.5324289670579...|  408.6403510726275|
|[30.4925366965402...|  282.4712457199145|
|[30.5743636841713...| 442.06441375806565|
|[30.7377203726281...|  461.7807421962299|
|[30.8162006488763...|   266.086340948469|
|[30.8364326747734...|  467.5019004269896|
|[30.8794843441274...|  490.2065999848547|
|[31.0613251567161...|  487.5554580579016|
|[31.1280900496166...|  557.2526867470547|
|[31.1695067987115...|  427.3565308022928|
|[31.2606468698795...|  421.3266312569514|
|[31.3091926408918...|  432.7207178399336|
|[31.3123495994443...|  463.5914180279406|
|[31.3584771924370...|  495.1759504494754|
|[31.3895854806643...|  410.0696110599829|
|[31.4252268808548...|  530.7667186547619|
|[31.5147378578019...|  489.8124879964614|
|[31.5171218025062...|  275.9184206503857|
|[31.5257524169682...|  443.9656268098819|
|[31.5261978982398...|  409.0945261923378|
+----------

In [21]:
# Training out linerRegression Model
lr = LinearRegression(featuresCol='features', labelCol='Yearly Amount Spent')

lrModel = lr.fit(trainingData)

22/12/03 19:37:45 WARN Instrumentation: [d77c2e51] regParam is zero, which might cause numerical instability and overfitting.
22/12/03 19:37:45 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
22/12/03 19:37:45 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/12/03 19:37:45 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/12/03 19:37:45 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [22]:
modelSummary = lrModel.summary

In [26]:
# Evaluating on our testData
testResults = lrModel.evaluate(testData)

# Residuals is the diff b/w predicted value and testData
testResults.residuals.show(5)

+-------------------+
|          residuals|
+-------------------+
|-10.298150708816024|
|  6.945428837500458|
|  4.487854399554294|
|-12.699300504089422|
| -21.00857806625129|
+-------------------+
only showing top 5 rows



In [28]:
print(testResults.rootMeanSquaredError)
testResults.r2

10.21850673650361


0.9829292435958727

In [30]:
unlabeled_data = outputData.select('features')

In [33]:
# Making predictions
predictedData = lrModel.transform(unlabeled_data)

In [34]:
predictedData.show(5)

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[34.4972677251122...| 595.5778485854466|
|[31.9262720263601...|379.90960484875995|
|[33.0009147556426...|  505.516431975522|
|[34.3055566297555...| 569.6860737023314|
|[33.3306725236463...| 591.9004594723935|
+--------------------+------------------+
only showing top 5 rows

